<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/Prompt_Caching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install bs4 anthropic

In [3]:
import getpass
import os

import anthropic
import time
import requests
from bs4 import BeautifulSoup

In [4]:
if not os.environ.get("ANTHROPIC_API_KEY"):
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("ANTHROPIC API Key:")

ANTHROPIC API Key:··········


# 利用文章の取得

In [5]:
def fetch_article_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Remove script and style elements
    for script in soup(["script", "style"]):
        script.decompose()

    # Get text
    text = soup.get_text()

    # Break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # Break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # Drop blank lines
    text = "\n".join(chunk for chunk in chunks if chunk)

    return text


# Fetch the content of the article
book_url = "https://www.gutenberg.org/cache/epub/1342/pg1342.txt"
book_content = fetch_article_content(book_url)

print(f"Fetched {len(book_content)} characters from the book.")
print("First 500 characters:")
print(book_content[:500])

Fetched 737522 characters from the book.
First 500 characters:
The Project Gutenberg eBook of Pride and Prejudice
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.
Title:


In [6]:
book_content_part = book_content[:4000]

In [7]:
book_content_part

'The Project Gutenberg eBook of Pride and Prejudice\nThis ebook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this ebook or online\nat www.gutenberg.org. If you are not located in the United States,\nyou will have to check the laws of the country where you are located\nbefore using this eBook.\nTitle: Pride and Prejudice\nAuthor: Jane Austen\nRelease date: June 1, 1998 [eBook #1342]\nMost recently updated: June 17, 2024\nLanguage: English\nCredits: Chuck Greif and the Online Distributed Proofreading Team at http://www.pgdp.net (This file was produced from images available at The Internet Archive)\n*** START OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE ***\n[Illustration:\nGEORGE ALLEN\nPUBLISHER\n156 CHARING CROSS ROAD\nLONDON\nRUSKIN HOUSE\n]\n[Illustration:\n_Reading Jane’s 

# API呼び出し  
参考:https://github.com/anthropics/anthropic-cookbook/blob/main/misc/prompt_caching.ipynb

In [8]:
client = anthropic.Anthropic()
def api_call():
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "<book>" + book_content_part + "</book>",
                    "cache_control": {"type": "ephemeral"},
                },
                {
                    "type": "text",
                    "text": "What is the title of this book? Only output the title.",
                },
            ],
        }
    ]

    start_time = time.time()
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=300,
        messages=messages,
        extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"},
    )
    end_time = time.time()
    return response, end_time - start_time

In [9]:
response, elapsed_time = api_call()

In [10]:
response.usage

Usage(input_tokens=17, output_tokens=8, cache_creation_input_tokens=1107, cache_read_input_tokens=0)

In [14]:
response.content[0].text

'Pride and Prejudice'

In [15]:
elapsed_time

1.1239683628082275

In [16]:
response, elapsed_time = api_call()

In [17]:
response.usage

Usage(input_tokens=17, output_tokens=8, cache_creation_input_tokens=0, cache_read_input_tokens=1107)

In [18]:
elapsed_time

1.5080997943878174